# Etiquetado de Tópicos Predefinidos con LLM Multi-Etiqueta

Este notebook implementa un sistema de clasificación multi-etiqueta para reseñas turísticas utilizando un LLM con salida estructurada. El objetivo es etiquetar cada reseña con categorías predefinidas para posteriormente entrenar modelos BERT o similares.

## Import Required Libraries

In [ ]:
import pandas as pd
from typing import List
from pydantic import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
import time
import os

## Setup LLM and Pydantic Models

In [ ]:
class MultiLabelOutput(BaseModel):
    labels: List[int] = Field(..., description="Lista de números de categorías aplicables (0-14)")
    
    @validator('labels')
    def validate_labels(cls, v):
        valid_labels = set(range(15))
        
        if not all(label in valid_labels for label in v):
            raise ValueError("Todas las etiquetas deben estar entre 0 y 14")
        
        if 14 in v and len(v) > 1:
            raise ValueError("La categoría 14 (Otros) solo puede aparecer sola")
        
        return sorted(list(set(v)))

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

parser = PydanticOutputParser(pydantic_object=MultiLabelOutput)

## Load and Prepare Dataset

In [ ]:
df = pd.read_csv('../data/processed/dataset_opiniones_analisis.csv')

print(f"Dataset cargado: {len(df)} reseñas")
print(f"Primeras columnas: {list(df.columns[:5])}")
print(f"Ejemplo de TituloReview: {df['TituloReview'].iloc[0][:100]}...")

df_sample = df.copy()

## Create Classification Prompt

In [ ]:
prompt_template = """
Eres un experto en el sector turístico y en etiquetado de datos. Tu tarea es clasificar reseñas turísticas en categorías predefinidas usando etiquetado multi-etiqueta.

CATEGORÍAS DISPONIBLES:
0. Alojamiento - Hoteles, resorts, hospedaje, habitaciones, servicios de hotel
1. Gastronomía - Comida, restaurantes, bebidas, experiencias culinarias
2. Transporte - Medios de transporte, taxis, autobuses, pulmonías, accesibilidad
3. Eventos y festivales - Carnaval, festivales, eventos especiales, espectáculos
4. Historia y cultura - Patrimonio histórico, museos, cultura local, tradiciones
5. Compras - Tiendas, mercados, artesanías, vendedores ambulantes
6. Deportes y aventura - Actividades deportivas, aventura, clavadistas
7. Vida nocturna - Bares, entretenimiento nocturno, ambiente festivo
8. Naturaleza y actividades al aire libre - Paseos, caminatas, actividades en exteriores
9. Playas y actividades acuáticas - Playas, océano, actividades marítimas, natación
10. Personal y servicio - Atención al cliente, amabilidad del personal, servicios
11. Seguridad - Aspectos de seguridad, criminalidad, protección
12. Costo/Precio - Precios, valor por dinero, costos, presupuesto
13. Ambiente/entorno - Ambiente general, atmósfera, limpieza, ruido
14. Otros - Solo usar cuando NINGUNA de las categorías anteriores aplique

INSTRUCCIONES:
- Cada reseña puede tener MÚLTIPLES etiquetas donde sea apropiado
- La categoría 14 (Otros) SOLO puede ser elegida si NINGUNA otra categoría es aplicable
- Si eliges la categoría 14, NO puedes elegir ninguna otra
- Devuelve SOLO los números de las categorías como lista: [0, 1, 5]
- Si realmente no encuentras ninguna categoría aplicable, puedes usar [14]

RESEÑA A CLASIFICAR:
{review_text}

{format_instructions}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["review_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | llm | parser

## Implement Batch Processing Function

In [ ]:
def classify_review(review_text):
    try:
        result = chain.invoke({"review_text": review_text})
        return result.labels
    except Exception as e:
        print(f"Error procesando reseña: {str(e)[:100]}")
        return [14]

def process_reviews_batch(df, start_idx=0, batch_size=50):
    results = []
    total = len(df)
    
    for i in range(start_idx, total):
        if i % batch_size == 0:
            print(f"Procesando {i}/{total} reseñas...")
        
        review = df.iloc[i]['TituloReview']
        labels = classify_review(review)
        results.append(labels)
        
        time.sleep(0.5)
    
    return results

## Process Dataset Reviews

In [ ]:
print("Clasificando primeras 5 reseñas como prueba...")
test_labels = []
for i in range(5):
    review = df_sample.iloc[i]['TituloReview']
    labels = classify_review(review)
    test_labels.append(labels)
    print(f"Reseña {i+1}: {review[:50]}... -> {labels}")
    time.sleep(1)

print("\nProcesando todo el dataset...")
all_labels = process_reviews_batch(df_sample)

df_sample['EtiquetasMultiples'] = all_labels

category_names = [
    'Alojamiento', 'Gastronomía', 'Transporte', 'Eventos y festivales', 
    'Historia y cultura', 'Compras', 'Deportes y aventura', 'Vida nocturna',
    'Naturaleza y actividades al aire libre', 'Playas y actividades acuáticas',
    'Personal y servicio', 'Seguridad', 'Costo/Precio', 'Ambiente/entorno', 'Otros'
]

for i, category in enumerate(category_names):
    df_sample[f'Categoria_{i}_{category.replace(" ", "_").replace("/", "_")}'] = df_sample['EtiquetasMultiples'].apply(lambda x: 1 if i in x else 0)

## Save Results

In [ ]:
output_path = '../data/processed/dataset_opiniones_etiquetado_multietiqueta.csv'
df_sample.to_csv(output_path, index=False)

print(f"Dataset etiquetado guardado en: {output_path}")
print(f"Total de reseñas procesadas: {len(df_sample)}")

label_counts = {}
for labels_list in df_sample['EtiquetasMultiples']:
    for label in labels_list:
        label_counts[label] = label_counts.get(label, 0) + 1

print("\nDistribución de etiquetas:")
for i, count in sorted(label_counts.items()):
    category_name = category_names[i]
    print(f"{i}: {category_name} -> {count} reseñas")

multi_label_count = sum(1 for labels in df_sample['EtiquetasMultiples'] if len(labels) > 1)
print(f"\nReseñas con múltiples etiquetas: {multi_label_count}/{len(df_sample)} ({multi_label_count/len(df_sample)*100:.1f}%)")

sample_multi = df_sample[df_sample['EtiquetasMultiples'].apply(len) > 1].head(3)
print("\nEjemplos de reseñas multi-etiqueta:")
for idx, row in sample_multi.iterrows():
    print(f"'{row['TituloReview'][:60]}...' -> {row['EtiquetasMultiples']}")